Poppy humanoid robot is a humanoid robot that can be used with its hardware platform or with the CoppeliaSim simulator. It can be programmed with python, using the pypot library.

Download and install the simulator CoppeliaSim from https://www.coppeliarobotics.com/downloads 

# Setup the software

Poppy uses pypot for control. It is a python library : http://poppy-project.github.io/pypot/. 
On the top of pypot are libraries for Poppy creatures : https://github.com/poppy-project.
The quick install consists in:
- install pypot :

!py -m pip install pypot

- install your poppy creature with its geometry :


In [2]:
#!py -m pip install poppy_torso

- install the library **ikpy** that proposes the inverse kinematics of the robot. The source code is https://github.com/Phylliade/ikpy. See tutorial on https://notebook.community/Phylliade/ikpy/tutorials/Moving%20the%20Poppy%20Torso%20using%20Inverse%20Kinematics)

In [3]:
#!py -m pip install 'ikpy[plot]'

We use another algorithm for human pose estimation : Blazepose :
- the article describing the algorithm is in https://arxiv.org/abs/2006.10204
- the source code is still available at https://github.com/google/mediapipe

To install, use the command : 

In [4]:
#!pip install mediapipe

# Start your code
Now you are done with the installation phase. You can start your project by importing the different libraries

In [5]:
import numpy as np
from utils.skeleton import *
from utils.quaternion import *
from utils.blazepose import blazepose_skeletons
import os
from pypot.creatures import PoppyTorso
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
from pypot.creatures.ik import IKChain
from pypot.primitive.move import Move
from pypot.primitive.move import MovePlayer

## Capture a video with a camera

For this section only, you need to install first opencv.

In [6]:
#import cv2
#import time

#time.sleep(5)

# Open a connection to the cameras
#cap1 = cv2.VideoCapture(0)

Capture the frames of the camera for 10 seconds. Save as file cam1.avi

In [7]:
#frame_width_1 = int(cap1.get(3))
#frame_height_1 = int(cap1.get(4))
# if you want to change the resolution of the camera
#cap1.set(3,frame_width_1)
#cap1.set(4,frame_height_1)


#size_1 = (frame_width_1, frame_height_1)

#video_nom_1 = "./resources/cam1.avi"

# Compression
#lossless = cv2.VideoWriter_fourcc(* 'FFV1')

# Save the frame as an video file
#video_1 = cv2.VideoWriter(video_nom_1, lossless, 30, size_1)

#start = time.time()

#while  time.time() < start + 10:

    # Capture a frame from the cameras
#    ret1, frame1 = cap1.read()

    # Check if the user pressed the enter key
#    cv2.waitKey(1)

#    if ret1:
        #cv2.imshow("Webcam 1", frame1) #to display the camera images
#        video_1.write(frame1)


#video_1.release()

# Release the camera and close the window
#cap1.release()
#cv2.destroyAllWindows()

# Instantiate the robot

Now open the simulator CoppeliaSim. Click yes when prompted if you accept all incoming communications.
For MacOs, type in the Sandbox script terminal

simExtRemoteApiStart(19997)

If instead it does now appear and you get an error message like


> pypot.vrep.io.VrepConnectionError: Could not connect to V-REP server on 127.0.0.1:19997. This could also means that you still have a previously opened connection running! (try pypot.vrep.close_all_connections())
>
> During handling of the above exception, another exception occurred:
>
> pypot.vrep.io.VrepIOErrors: No value

type in the Sandbox script terminal

simExtRemoteApiStart(19997)
    

Instantiate the poppy robot in the simulator with the code below, this is necessary in order to add the kinematic chains attributes.

In [8]:
from pypot import vrep
vrep.close_all_connections()
poppy = PoppyTorso(simulator='vrep')

KeyboardInterrupt: 

Poppy should now appear on the CoppeliaSim simulation screen, and a popup appeared in CoppeliaSim to inform you that the simulation use custom parameters. This popup block the communication to the Python API of CoppeliaSim. You have to check the check-box “Do not show this message again” and press “Ok”. Do this 3 times.

In [ ]:
t = None
targets = None
smoothed_targets = None

For each kinematic chain we have built an urdf file. We create an IKChain object for each kinematic chain, making it possible to compute the inverse kinematics, i.e. motor angles from desired end-effector position.

The constructor takes as input the poppy robot instance, the motors that are part of the kinematic chain, the motors that remain passive during the inverse kinematics, the distance of the tip of the last bone of the chain, and finally, the list of motors for which the urdf file give reversed orientation.

We can plot some of these kinematic chains in a figure. If the position of the robot in the simulator is changed, these changes should be reflected when reexecuting this cell.

In [ ]:
import matplotlib
from mpl_toolkits.mplot3d import Axes3D
%matplotlib notebook
zero = [0] * 7

ax = matplotlib.pyplot.figure().add_subplot(111, projection='3d')
ax.scatter([0], [0],[0])

poppy.l_arm_chain.plot(poppy.l_arm_chain.convert_to_ik_angles(poppy.l_arm_chain.joints_position), ax, target = (0.2, -0.2, 0.2))
poppy.r_arm_chain.plot(poppy.r_arm_chain.convert_to_ik_angles(poppy.r_arm_chain.joints_position), ax)


<IPython.core.display.Javascript object>

The robot is now ready to use.

# Using the robot Poppy



You can directly access the chains:


In [ ]:
print("chaine  " , poppy.l_arm_chain)
print("nom     " , poppy.l_arm_chain.name)
print("links   " , poppy.l_arm_chain.links)
print("1er link" , poppy.l_arm_chain.links[1].name)
#print(poppy.torso_chain)
#print(poppy.l_elbow_chain   )
#print(poppy.l_elbow_chain   )


NameError: name 'poppy' is not defined

You can access their respective motors

In [ ]:
[m.name for m in poppy.l_arm_chain.motors]

['abs_z',
 'bust_y',
 'bust_x',
 'l_shoulder_y',
 'l_shoulder_x',
 'l_arm_z',
 'l_elbow_y']

You can access the state of the robot.

joints_position returns the joint angles for all the motors of the chain

In [ ]:
print(poppy.l_arm_chain.joints_position)

[-0.0, 3.1, -0.6, 65.0, 194.9, -36.8, 32.6]


position returns the cartesian position of the end effector

In [ ]:
print(poppy.l_arm_chain.position)

[ 0.14207496 -0.29099236  0.21244525]


In [ ]:
poppy.l_shoulder_y.goto_position(-30,3)
poppy.l_shoulder_x.goto_position(30,3)
poppy.abs_z.goto_position(-20,3)
print(poppy.l_arm_chain.position)

[ 0.14207496 -0.29099236  0.21244525]


Reset the robot to an initial position

In [ ]:
def poppy_reset():
    joint_pos = { 'l_elbow_y':0.0,
                 'head_y': 0.0,
                 'r_arm_z': 0.0,
                 'head_z': 0.0,
                 'r_shoulder_x': 0.0,
                 'r_shoulder_y': 0.0,
                 'r_elbow_y': 0.0,
                 'l_arm_z': 0.0,
                 'abs_z': 0.0,
                 'bust_y': 0.0,
                 'bust_x':0.0,
                 'l_shoulder_x': 0.0,
                 'l_shoulder_y': 0.0
                }
    for m in poppy.motors:
        m.goto_position(joint_pos[m.name],5)

poppy_reset()

In [ ]:
from pypot.primitive.move import Move
fps =10

move = Move(freq=fps)


print("list of all motors of poppy", [m.name for m in poppy.motors])
move_motors = [m.name for m in poppy.motors]


for t in np.linspace(0.02,3,int(3*fps)):
        new_positions = {}
        for motor in move_motors:
            # decide for each timestep and each motor a joint angle and a velocity
            new_positions[motor] = [20*np.sin(t), 0.0]

        move.add_position(new_positions, t)

#print("joint positions of the move ",(move._timed_positions))

list of all motors of poppy ['l_elbow_y', 'head_y', 'r_arm_z', 'head_z', 'r_shoulder_x', 'r_shoulder_y', 'r_elbow_y', 'l_arm_z', 'abs_z', 'bust_y', 'bust_x', 'l_shoulder_x', 'l_shoulder_y']


Before sending the motor commands to the robot, reset the robot to an initial position

In [ ]:
poppy_reset()

Send the motor commands to the robot

In [ ]:
mp = MovePlayer(poppy, move,play_speed=1)
mp.start()

Record the movement in an file

In [ ]:
move.save(open('new_movement.record', 'w'))

# Imitation by inverse kinematics

This function is a wrapper for the inverse kinematics methods of the IKChain objects.

If no initial position is provided, the method will use the current position of the robot in the simulator, and will automatically control the simulated robot towards the provided target.

It returns the new joints angle coordinates.

In [ ]:
def ik(kinematic_chain, target_position, initial_position=None):

    kwargs = {}
    kwargs['max_iter'] = 3
    if initial_position is not None:
        kwargs['initial_position'] = kinematic_chain.convert_to_ik_angles(initial_position)
    else:
        kwargs['initial_position'] = kinematic_chain.convert_to_ik_angles(kinematic_chain.joints_position)

    q = kinematic_chain.inverse_kinematics(
        target_position=target_position,
        orientation_mode=None,
        **kwargs
    )

    joints = kinematic_chain.convert_from_ik_angles(q)

    last = kinematic_chain.motors[-1]

    if initial_position is None:
        for i, (m, pos) in enumerate(list(zip(kinematic_chain.motors, joints))):
            if kinematic_chain.active_links_mask[i+1]:
                m.goal_position = pos

    return joints

## Pose estimation

This function uses blazepose to compute the skeleton based on a video file.
Blazepose/Mediapipe is a 3D human pose estimation algorithm that can run in realtime on a CPU computer. 

This function processes a video and returns a list of positions (x,y,z) for each joint.

In [ ]:
skeletons = blazepose_skeletons('./resources/mai1.mov')

Let's examine the skeletons variable. It is a tensor of size :
- the number of frames in the video
- 17 joints in the human figure model
- 3 for the (x,y,z) positions

In [ ]:
print(skeletons.shape)
print(skeletons)


torch.Size([279, 17, 3])
tensor([[[ 4.9751e-01,  8.3294e-01, -4.8693e-05],
         [ 5.2856e-01,  8.3197e-01, -5.8598e-03],
         [ 5.3034e-01,  1.0175e+00, -7.3543e-02],
         ...,
         [ 4.4473e-01,  5.9587e-01, -2.2271e-01],
         [ 4.2788e-01,  7.4048e-01, -1.7966e-01],
         [ 4.0426e-01,  8.6796e-01, -2.5938e-01]],

        [[ 4.9798e-01,  8.4524e-01,  2.4405e-05],
         [ 5.2971e-01,  8.4502e-01, -4.3802e-03],
         [ 5.3006e-01,  1.0522e+00, -1.6453e-02],
         ...,
         [ 4.4491e-01,  5.9534e-01, -1.9633e-01],
         [ 4.2767e-01,  7.4063e-01, -1.3409e-01],
         [ 4.0358e-01,  8.6794e-01, -2.0700e-01]],

        [[ 4.9807e-01,  8.5293e-01,  7.3847e-05],
         [ 5.3016e-01,  8.5253e-01, -1.9893e-03],
         [ 5.2929e-01,  1.0645e+00, -1.1190e-02],
         ...,
         [ 4.4505e-01,  5.9502e-01, -1.8995e-01],
         [ 4.2669e-01,  7.4064e-01, -1.3442e-01],
         [ 4.0196e-01,  8.6793e-01, -2.1391e-01]],

        ...,

        [[ 5.

Normalize the skeleton, change the reference frame

In [ ]:
def change_frame(skeletons, frame_name, alpha, topology):

    rota_skeletons_A = skeletons.clone()
    rota_skeletons_A[:, :, 2] = -skeletons[:, :, 1]
    rota_skeletons_A[:, :, 1] = skeletons[:, :, 2]
    center_A = rota_skeletons_A[:, 0,:].unsqueeze(1).repeat(1, len(topology), 1)
    rota_skeletons_A = rota_skeletons_A - center_A

    batch_size, n_joints, _ = rota_skeletons_A.shape


    # Measure skeleton bone lengths
    lengths = torch.Tensor(batch_size, n_joints)
    for child, parent in enumerate(topology):
            lengths[:, child] = torch.sqrt(
                torch.sum(
                    (rota_skeletons_A[:, child] - rota_skeletons_A[:, parent])**2,
                    axis=-1
                )
            )

    # Find the corresponding angles
    offsets = torch.zeros(batch_size, n_joints, 3)
    offsets[:, :, -1] = lengths
    quaternions = find_quaternions(topology, offsets, rota_skeletons_A)

    # Rotate of alpha
    #define the rotation by its quaternion
    rotation = torch.Tensor([np.cos(alpha/2),  np.sin(alpha/2),0,0]).unsqueeze(0).repeat(batch_size*n_joints, 1)
    quaternions = quaternions.reshape(batch_size*n_joints, 4)
    quaternions = batch_quat_left_multiply(
            batch_quat_inverse(rotation),
            quaternions
        )
    quaternions = quaternions.reshape(batch_size, n_joints, 4)

    # Use these quaternions in the forward kinematics with the Poppy skeleton
    skeleton = forward_kinematics(
            topology,
            torch.zeros(batch_size, 3),
            offsets,
            quaternions
        )[0]

    outputs= skeleton.clone()

    return outputs

In [ ]:
# Skeleton topology
topology = [0, 0, 1, 2, 0, 4, 5, 0, 7, 8, 9, 8, 11, 12, 8, 14, 15]

In [ ]:
alpha = np.pi/4.

rota_skeletons_B = change_frame(skeletons, 'general', alpha, topology)

Plot the skeleton in the new reference

In [ ]:
%matplotlib notebook
print(rota_skeletons_B.shape)
t=10

ax = pyplot_skeleton(topology, rota_skeletons_B[t], show=True, color='blue') #output by blazepose
#ax=pyplot_skeleton(topology, rota_skeletons_B[t], ax=ax, show=True, color='red') #in the new reference

torch.Size([279, 17, 3])


<IPython.core.display.Javascript object>

Then, this function computes target XYZ positions for Poppy's kinematic chains' end effectors based on the skeleton obtained from blazepose. It proceeds as follows:
- it estimates the source (i.e. video) bone lengths
- it estimates the source (i.e. video) joint orientations using the "find_quaternions" util function
- it reorients all the joint angles using as base orientation the pelvis -> chest axis
- it computes the XYZ joint positions based on the found orientations and the poppy bone lengths

A little trick is then applied, but could be removed. To decrease the depth of the movement estimated by blazepose, the y-axis values are divided by 10. To ensure that the target positions still correspond to achievable positions by the robot, we do another round of XYZ positions -> joint orientations -> XYZ positions.

Finally, this function returns the joint XYZ positions only for kinematic chaine end effectors.

In [ ]:
def targets_from_skeleton(source_positions, topology):
    # Works in batched
    batch_size, n_joints, _ = source_positions.shape

    # Measure skeleton bone lengths
    source_lengths = torch.Tensor(batch_size, n_joints)
    for child, parent in enumerate(topology):
        source_lengths[:, child] = torch.sqrt(
            torch.sum(
                (source_positions[:, child] - source_positions[:, parent])**2,
                axis=-1
            )
        )

    # Find the corresponding angles
    source_offsets = torch.zeros(batch_size, n_joints, 3)
    source_offsets[:, :, -1] = source_lengths
    quaternions = find_quaternions(topology, source_offsets, source_positions)

    # Re-orient according to the pelvis->chest orientation
    base_orientation = quaternions[:, 7:8].repeat(1, n_joints, 1).reshape(batch_size*n_joints, 4)
    base_orientation += 1e-3 * torch.randn_like(base_orientation)
    quaternions = quaternions.reshape(batch_size*n_joints, 4)
    quaternions = batch_quat_left_multiply(
        batch_quat_inverse(base_orientation),
        quaternions
    )
    quaternions = quaternions.reshape(batch_size, n_joints, 4)

    # Use these quaternions in the forward kinematics with the Poppy skeleton
    target_offsets = torch.zeros(batch_size, n_joints, 3)
    target_offsets[:, :, -1] = poppy_lengths.unsqueeze(0).repeat(batch_size, 1)
    target_positions = forward_kinematics(
        topology,
        torch.zeros(batch_size, 3),
        target_offsets,
        quaternions
    )[0]

    # Measure the hip orientation
    alpha = np.arctan2(
        target_positions[0, 1, 1] - target_positions[0, 0, 1],
        target_positions[0, 1, 0] - target_positions[0, 0, 0]
    )

    # Rotate by alpha around z
    alpha = alpha
    rotation = torch.Tensor([np.cos(alpha/2), 0, 0, np.sin(alpha/2)]).unsqueeze(0).repeat(batch_size*n_joints, 1)
    quaternions = quaternions.reshape(batch_size*n_joints, 4)
    quaternions = batch_quat_left_multiply(
        batch_quat_inverse(rotation),
        quaternions
    )
    quaternions = quaternions.reshape(batch_size, n_joints, 4)

    # Use these quaternions in the forward kinematics with the Poppy skeleton
    target_positions = forward_kinematics(
        topology,
        torch.zeros(batch_size, 3),
        target_offsets,
        quaternions
    )[0]



    # Return only target positions for the end-effector of the 6 kinematic chains:
    # Chest, head, left hand, left elbow, left shoulder, right hand, right elbow
    # end_effector_indices = [8, 10, 13, 12, 11, 16, 15]
    end_effector_indices = [13, 16]
    # end_effector_indices = [13, 12, 16, 15]

    return target_positions[:, end_effector_indices], target_positions

In [ ]:
poppy_lengths = torch.Tensor([
    0.0,
    0.07,
    0.18,
    0.19,
    0.07,
    0.18,
    0.19,
    0.12,
    0.08,
    0.07,
    0.05,
    0.1,
    0.15,
    0.13,
    0.1,
    0.15,
    0.13
])

In [ ]:
targets, all_positions = targets_from_skeleton(skeletons, topology)

We can display the target skeleton with stars representing the end-effector target positions.

In [ ]:
%matplotlib notebook

t = t if t is not None else 0
t = -100

ax = pyplot_skeleton(topology, all_positions[t], show=False)
ax.scatter(targets[t, :, 0], targets[t, :, 1], targets[t, :, 2], c='red', s=500, marker='*')

ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
set_axes_equal(ax)
plt.show()

<IPython.core.display.Javascript object>

For smoother movements, we compute a moving average of the target positions.

In [ ]:
def moving_average(a, n=3) :
    repeat_shape = list(a.shape)
    repeat_shape[1:] = [1 for _ in range(len(repeat_shape)-1)]
    repeat_shape[0] = n//2
    a = torch.cat([a[:1].repeat(*repeat_shape), a, a[-2:].repeat(*repeat_shape)])
    ret = torch.cumsum(a, axis=0)
    ret[n:] = ret[n:] - ret[:-n]
    return ret[n - 1:] / n

In [ ]:
# Can be used to have more keypoints for the inverse kinematics. It could be useful for fast movements.
# With factor=1, it does nothing
def interpolate_targets(targets, factor=1):

    length, joints, _ = targets.shape

    new_targets = torch.zeros((length-1) * factor + 1, joints, 3)

    for i in range(new_targets.shape[0]):

        target_id = float(i/factor)
        before_id = int(np.floor(target_id))
        after_id = int(np.floor(target_id + 1))

        before_coef = 1 - (target_id - before_id)
        after_coef = 1 - (after_id - target_id)

        if after_id > length - 1:
            after_id = length - 1

        new_targets[i] = before_coef * targets[before_id] + after_coef * targets[after_id]

    return new_targets

In [ ]:
targets.shape

torch.Size([279, 2, 3])

In [ ]:
interpolated_targets = interpolate_targets(targets)
smoothed_targets = moving_average(interpolated_targets, n=15)

In [ ]:
plt.figure()
plt.plot(targets[:, :, 0])
plt.suptitle('Raw Targets x')
plt.show()

<IPython.core.display.Javascript object>

In [ ]:
plt.figure()
plt.plot(targets[:, :, 1])
plt.suptitle('Smoothed Targets x')
plt.show()

<IPython.core.display.Javascript object>

In [ ]:
plt.figure()
plt.plot(smoothed_targets[:, :, 0])
plt.suptitle('Smoothed Targets y')
plt.show()

<IPython.core.display.Javascript object>

In [ ]:
plt.figure()
plt.plot(smoothed_targets[:, :, 1])
plt.suptitle('Smoothed Targets z')
plt.show()

<IPython.core.display.Javascript object>

These two functions perform inverse kinematics combining multiple kinematic chains at the same time. The upper_body_imitation uses torso, head, elbows and hands targets, while the arms_imitation only focuses on the two hands.

If positions are provided, its uses it as initial positions, otherwise, it uses the current simulated robot configuration.

In [ ]:
import time

In [ ]:
def dict_from_position(kinematic_chains, positions):

    results = {}

    for kc, joint_positions in zip(kinematic_chains, positions):

        for i, (motor, motor_position) in enumerate(zip(kc.motors, joint_positions)):

            if motor.name not in enumerate(results.keys()) and kc.active_links_mask[i+1]:
                results[motor.name] = [motor_position, 0.]

    return results

In [ ]:
def upper_body_imitation(poppy, targets, positions=None):

    # Targets should be a tensor of shape (6, 3)
    # In order: chest, head, r_hand, l_hand, l_elbow, r_elbow
    kinematic_chains = [
        #poppy.torso_chain,
        #poppy.head_chain,
        #poppy.l_shoulder_chain,
        poppy.l_arm_chain,
        #poppy.l_hand_chain,
        poppy.r_arm_chain,
    ]

    next_positions = []

    for i, kinematic_chain in enumerate(kinematic_chains):

        if positions is not None:
            next_positions.append(ik(kinematic_chain, targets[i], positions[i]))
        else:
            next_positions.append(ik(kinematic_chain, targets[i]))

    return next_positions

In [ ]:
import copy
from pypot.primitive.move import Move

This cell loops along the different frames of the input video and performs frame by frame imitation. It registers the found motor angles in a Move object that stores the whole Poppy movement.

In [ ]:
poppy_reset()

In [ ]:
fps = 10

for motor in poppy.motors:
    motor.compliant = False

# Upper body imitation seems to work better
# Otherwise, the elbow might go to wrong positions that can block the motion later

kinematic_chains = [
    # poppy.torso_chain,
    #poppy.head_chain,
    #poppy.l_shoulder_chain,
    poppy.l_arm_chain,
    #poppy.l_elbow_chain,
    # poppy.l_hand_chain,
    poppy.r_arm_chain,
    #poppy.r_elbow_chain,
    # poppy.r_hand_chain,
]

#kinematic_chains = [
#    poppy.l_arm_chain,
#    poppy.r_arm_chain,
#]

positions = [k.joints_position for k in kinematic_chains]

move = Move(freq=fps)

for t in range(smoothed_targets.shape[0]):
    time.sleep(1./fps)
    #positions = upper_body_imitation(
    #    poppy,
    #    smoothed_targets[t],
    #    # positions = positions
    #)

    positions = upper_body_imitation(
        poppy,
        smoothed_targets[t],
    )

    move.add_position(
        dict_from_position(kinematic_chains, positions),
        float(t)/fps
    )

AttributeError: module 'numpy' has no attribute 'float'.
`np.float` was a deprecated alias for the builtin `float`. To avoid this error in existing code, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
The aliases was originally deprecated in NumPy 1.20; for more details and guidance see the original release note at:
    https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations

We can plot the evolution of the motor angles during the trajectory. 

In [ ]:
%matplotlib notebook

ax = plt.axes()
move.plot(ax)
plt.show()

save this movement to a file

In [ ]:
move.save(open('move.record', 'w'))

## Play the saved move in the simulator

In [ ]:
poppy_reset()

In [ ]:
from pypot.primitive.move import MovePlayer, MoveRecorder

In [ ]:
def smoothen_move(move):

    # Function to smoothen the Poppy movement
    n = 10

    # Create a tensor from the dictionary
    motors = move.positions()[0].keys()
    move_tensor = torch.Tensor([
        [move.positions()[t][motor]  for motor in motors] for t in move.positions().keys()
    ])

    # Control the motor range
    move_tensor = torch.minimum(move_tensor, torch.full(move_tensor.shape, 180.))
    move_tensor = torch.maximum(move_tensor, torch.full(move_tensor.shape, -180.))

    # Moving average to smoothen the positions
    move_tensor = moving_average(move_tensor, n=n)

    # Compute velocity as the (next position - previous positions) * fps / 2
    move_tensor[1:-1, :, 1] = (move_tensor[2:, :, 0] - move_tensor[:-2, :, 0]) * 0.5 * move.framerate

    # Rebuild the dictionary from the tensor
    new_move = Move(freq=move.framerate)

    for i in range(move_tensor.shape[0]):
        dictionary = {}
        for j, motor in enumerate(motors):
            dictionary[motor] = move_tensor[i, j].tolist()
        new_move.add_position(
            dictionary,
            float(i)/fps
        )

    return new_move

In [ ]:
new_move = smoothen_move(move)

In [ ]:
mp = MovePlayer(poppy, new_move)

In [ ]:
mp.start()

In [ ]:
%matplotlib notebook

ax = plt.axes()
new_move.plot(ax)
plt.show()

Finally, we save the move in a file, copy the file through ssh to the poppy robot, and send an API request to play the move. Note that the computer executing the notebook should be on the same network than poppy.

# Your project

Your task is to use the reinforcement learning and inverse reinforcement learning algorithms to learn how Poppy can imitate the movement, without using the inverse kinematics library.

You can use stable baselines or imitation libraries. You can also use your own implementation of the RL or iRL algorithm.
For using the libraries, it is recommended that you create your own gymnamsium environment (NB gymnasium is the new version of gym, it can be used in the previous code by replacing the import line by 'import gymnasium as gym').

- You can refer to the documentation by gym on https://gymnasium.farama.org/tutorials/gymnasium_basics/environment_creation/ or https://www.gymlibrary.dev/content/environment_creation/. They propose a tutorial python notebook
- You can also look at an example project with a similar robot, ErgoJr : https://github.com/fgolemo/gym-ergojr with the gym environment implementation : https://github.com/fgolemo/gym-ergojr/blob/master/gym_ergojr/envs/ergo_reacher_env.py

The action is the joint positions given to each of the motors.
The observation are the cartesian positions that can be accessed by commands like poppy.l_arm_chain.position.


You also need to decide on the reward function to be used.

Two interesting articles to get your inspiration can be read : 

- https://arxiv.org/abs/2209.05135
- https://ieeexplore.ieee.org/stamp/stamp.jsp?arnumber=8956326&casa_token=10GKHz8KJR8AAAAA:OZNWV-X7RxXJqLlRNqMBEtBg7jbH4fyy8pjDiMf5cOT65USEECinEMOiEVj0VW5sUDETHjGVgA&tag=1

In [ ]:
import gymnasium as gym
from gymnasium import spaces
from imitation.algorithms import bc
import numpy as np
from stable_baselines3.common.vec_env import DummyVecEnv
from Poppy_Env_pierre import PoppyEnvPierre

env = PoppyEnvPierre()

print(env.observation_space)
print(env.action_space.shape)

Box([-0.2 -0.4 -0.2 -0.5 -0.4  0.2], [0.5 0.4 0.6 0.2 0.4 0.6], (6,), float32)
(39,)


In [ ]:
import torch

# Chemin vers les fichiers de données
path = './resources/anaele/'
file = 'anaele_1_poppy_skeletons.pt'
data_path = path + file

# Charger les données
poppy_skeletons = torch.load(data_path)
print("poppy_skeletons shape:", poppy_skeletons.shape)

observations = poppy_skeletons[:-1, :2, :].reshape(-1, 6)
print("Observations shape:", observations.shape)

actions = (poppy_skeletons[1:, :13, :] - poppy_skeletons[:-1, :13, :]).reshape(-1, 39)  # 13 joints * 3 composantes = 39
print("Actions shape:", actions.shape)

# Save formatted data for imitation learning
np.savez('formatted_training_data.npz', obs=observations.numpy(), acts=actions.numpy())

poppy_skeletons shape: torch.Size([149, 17, 3])
Observations shape: torch.Size([148, 6])
Actions shape: torch.Size([148, 39])


In [ ]:
import numpy as np
from imitation.data import types, rollout
from stable_baselines3 import PPO
from imitation.data import types

data = np.load('formatted_training_data.npz')

In [ ]:

observations = data['obs']
actions = data['acts']

max_action_value = np.max(actions, axis=None)
print("La valeur la plus élevée dans l'ensemble du tableau d'actions est :", max_action_value)

next_observations = observations[1:]
observations = observations[:-1]
actions = actions[:-1]

print("Next Observations shape:", next_observations.shape)
print("Observations shape:", observations.shape)
print("Actions shape:", actions.shape)

infos = [{} for _ in range(len(observations))]
dones = np.zeros(len(observations), dtype=bool)
dones[-1] = True
rews = np.zeros(len(observations))

transitions = types.TransitionsWithRew(
    obs=observations,
    acts=actions,
    next_obs=next_observations,
    infos=infos,
    dones=dones,
    rews=rews
)

policy_kwargs = {
    "net_arch": [dict(pi=[128, 128, 13], vf=[128, 128, 1])],
}

policy = PPO('MlpPolicy', env, learning_rate=0.001, gamma=0.99, ent_coef=100, verbose=1, batch_size=64, n_steps=2048, policy_kwargs=policy_kwargs)
print(policy.policy)

La valeur la plus élevée dans l'ensemble du tableau d'actions est : 0.06663132
Next Observations shape: (147, 6)
Observations shape: (147, 6)
Actions shape: (147, 39)
Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


c:\Projets\Poppy\.venv\lib\site-packages\stable_baselines3\common\policies.py:486: UserWarning: As shared layers in the mlp_extractor are removed since SB3 v1.8.0, you should now pass directly a dictionary and not a list (net_arch=dict(pi=..., vf=...) instead of net_arch=[dict(pi=..., vf=...)])
  warnings.warn(


ActorCriticPolicy(
  (features_extractor): FlattenExtractor(
    (flatten): Flatten(start_dim=1, end_dim=-1)
  )
  (pi_features_extractor): FlattenExtractor(
    (flatten): Flatten(start_dim=1, end_dim=-1)
  )
  (vf_features_extractor): FlattenExtractor(
    (flatten): Flatten(start_dim=1, end_dim=-1)
  )
  (mlp_extractor): MlpExtractor(
    (policy_net): Sequential(
      (0): Linear(in_features=6, out_features=128, bias=True)
      (1): Tanh()
      (2): Linear(in_features=128, out_features=128, bias=True)
      (3): Tanh()
      (4): Linear(in_features=128, out_features=13, bias=True)
      (5): Tanh()
    )
    (value_net): Sequential(
      (0): Linear(in_features=6, out_features=128, bias=True)
      (1): Tanh()
      (2): Linear(in_features=128, out_features=128, bias=True)
      (3): Tanh()
      (4): Linear(in_features=128, out_features=1, bias=True)
      (5): Tanh()
    )
  )
  (action_net): Linear(in_features=13, out_features=39, bias=True)
  (value_net): Linear(in_features

In [ ]:
rng = np.random.default_rng(seed=42)

model = bc.BC(
    observation_space=env.observation_space,
    action_space=env.action_space,
    rng=rng,
    policy=policy.policy,  # Adjust based on your requirements
    demonstrations=transitions,
    batch_size=128,  # Tune based on your computational resources
    optimizer_cls=torch.optim.Adam,  # Ensure you import torch.optim
    optimizer_kwargs={"lr": 0.0001},  # Example learning rate
    device="auto"  # Can be set to "cuda" or "cpu"
)

model.train(n_epochs=500)

0batch [00:00, ?batch/s]

--------------------------------
| batch_size        | 128      |
| bc/               |          |
|    batch          | 0        |
|    ent_loss       | -0.0553  |
|    entropy        | 55.3     |
|    epoch          | 0        |
|    l2_loss        | 0        |
|    l2_norm        | 283      |
|    loss           | 35.8     |
|    neglogp        | 35.8     |
|    prob_true_act  | 2.72e-16 |
|    samples_so_far | 128      |
--------------------------------


500batch [00:10, 45.92batch/s]


In [ ]:
print(env)
print(env.poppy)

<PoppyEnvPierre instance>
<Robot motors=[<DxlMotor name=l_elbow_y id=44 pos=0.0>, <DxlMotor name=head_y id=37 pos=0.0>, <DxlMotor name=r_arm_z id=53 pos=-0.0>, <DxlMotor name=head_z id=36 pos=-0.0>, <DxlMotor name=r_shoulder_x id=52 pos=0.0>, <DxlMotor name=r_shoulder_y id=51 pos=0.0>, <DxlMotor name=r_elbow_y id=54 pos=0.0>, <DxlMotor name=l_arm_z id=43 pos=0.0>, <DxlMotor name=abs_z id=33 pos=-0.0>, <DxlMotor name=bust_y id=34 pos=0.0>, <DxlMotor name=bust_x id=35 pos=0.0>, <DxlMotor name=l_shoulder_x id=42 pos=0.0>, <DxlMotor name=l_shoulder_y id=41 pos=0.0>]>


In [ ]:
policy.learn(total_timesteps=10)

JOINTS TO MOVE {'abs_z': -1.0743477, 'bust_y': -1.2503024, 'bust_x': -0.77974683, 'r_shoulder_x': 0.0, 'r_shoulder_y': 0.0, 'r_arm_z': 0.0, 'r_elbow_y': 1.9247903, 'l_shoulder_x': -0.23728181, 'l_shoulder_y': 0.4927716, 'l_arm_z': 0.0, 'l_elbow_y': 0.0, 'head_y': 0.0, 'head_z': 0.041982867}
JOINTS TO MOVE {'abs_z': 0.33584052, 'bust_y': -0.8605215, 'bust_x': 0.24867918, 'r_shoulder_x': 0.0, 'r_shoulder_y': 0.0, 'r_arm_z': 0.0, 'r_elbow_y': 0.9816317, 'l_shoulder_x': -0.19022681, 'l_shoulder_y': -0.37756395, 'l_arm_z': 0.0, 'l_elbow_y': 0.0, 'head_y': 0.0, 'head_z': -0.74981076}
JOINTS TO MOVE {'abs_z': 1.0225731, 'bust_y': 0.44491115, 'bust_x': -0.030111022, 'r_shoulder_x': 0.0, 'r_shoulder_y': 0.0, 'r_arm_z': 0.0, 'r_elbow_y': -1.0570344, 'l_shoulder_x': -0.093051076, 'l_shoulder_y': -1.2694725, 'l_arm_z': 0.0, 'l_elbow_y': 0.0, 'head_y': 0.0, 'head_z': -0.05176626}
JOINTS TO MOVE {'abs_z': -1.3692741, 'bust_y': 0.67336774, 'bust_x': 1.0328572, 'r_shoulder_x': 0.0, 'r_shoulder_y': 0.0

KeyboardInterrupt: 

In [2]:
from Poppy_Env_edouard_anaele import PoppyEnv
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3 import PPO
from stable_baselines3 import SAC
from imitation.algorithms import bc
import numpy as np
import torch
from imitation.data import types

device = "cuda" if torch.cuda.is_available() else "cpu"

env = PoppyEnv()
vec_env = DummyVecEnv([lambda: env])

rng = np.random.default_rng(seed=42)
poppy_skeletons = env.targets
observations = poppy_skeletons[:-1, :2, :].reshape(-1, 6)
actions = (poppy_skeletons[1:, :13, :] - poppy_skeletons[:-1, :13, :]).reshape(-1, 13)

next_observations = observations[1:]
observations = observations[:-1]
actions = actions[:-1]

infos = [{} for _ in range(len(observations))]
dones = np.zeros(len(observations), dtype=bool)
dones[-1] = True
rews = np.zeros(len(observations))

transitions = types.TransitionsWithRew(
    obs=observations,
    acts=actions,
    next_obs=next_observations,
    infos=infos,
    dones=dones,
    rews=rews
)

policy_kwargs = {
    "net_arch": [dict(pi=[512, 256, 128, 39], vf=[512, 256, 128, 1])],
}

learning_rate = 0.001  # Augmenter le taux d'apprentissage
clip_range = 100  # Augmenter la plage de clip
ent_coef = 100  # Augmenter le coefficient d'entropie
n_steps = 1024  # Augmenter le nombre d'étapes de roulement

# Créer le modèle avec les nouveaux paramètres
model = PPO("MlpPolicy", env, gae_lambda=0.9, max_grad_norm=100, ent_coef=ent_coef, learning_rate=learning_rate, clip_range=clip_range,
            n_steps=n_steps, verbose=1, device=device, policy_kwargs=policy_kwargs)

imitation = bc.BC(
    observation_space=env.observation_space,
    action_space=env.action_space,
    rng=rng,
    policy=model.policy,
    demonstrations=transitions,
    batch_size=128,
    optimizer_cls=torch.optim.Adam,
    optimizer_kwargs={"lr": 0.0001},
    device="auto"
)

imitation.train(n_epochs=500)

def evaluate_model(model, env, num_episodes=10):
    episode_rewards = []
    for _ in range(num_episodes):
        obs = env.reset()
        episode_reward = 0
        done = False
        while not done:
            action, _ = model.predict(obs, deterministic=True)
            obs, reward, done, _ = env.step(action)
            episode_reward += reward
        episode_rewards.append(episode_reward)
    return episode_rewards

model.learn(total_timesteps=10)

# Sauvegarder le modèle
model.save("ppo_poppy_model")

# Évaluer le modèle après l'entraînement
mean_rewards = evaluate_model(model, vec_env)

# Tracer l'évolution de la récompense moyenne
plt.plot(mean_rewards)
plt.xlabel('Episode')
plt.ylabel('Mean Reward')
plt.title('Evolution of Mean Reward')
plt.show()

Hello, I am Poppy!


AttributeError: 'PoppyEnv' object has no attribute 'targets'

In [ ]:
import torch
import numpy as np

def evaluate_model_2(model, env, num_episodes=3):
    episode_rewards = []
    model.policy.eval()  # Make sure the model is in evaluation mode.

    for _ in range(num_episodes):
        obs = env.reset()
        episode_reward = 0
        done = False
        while not done:
            with torch.no_grad():  # Disable gradient calculations
                # Check if obs is a tuple and extract the observation data
                obs_data = obs[0] if isinstance(obs, tuple) else obs
                print("Obs data:", obs_data)  # Debug statement
                # Check if obs_data is not None before creating the tensor
                if obs_data is not None:
                    obs_tensor = torch.tensor([obs_data], dtype=torch.float32)  # Add batch dimension

                    # Get action from the policy network
                    policy_output = model.policy(obs_tensor)
                    print(policy_output)
                    # Check if the output is a tuple (some policies return action and additional info)
                    if isinstance(policy_output, tuple):
                        action_tensor = policy_output[0]  # Assuming the action tensor is the first element
                    else:
                        action_tensor = policy_output

                    action = action_tensor.squeeze(0).numpy()  # Convert to numpy array and remove batch dimension
                    action = action
                else:
                    # Handle the case where obs_data is None
                    # You can choose to skip this step or handle it differently based on your requirements
                    # Here, I'm just printing a message indicating that obs_data is None
                    print("Warning: Observation data is None. Skipping step.")
                    continue

            obs, reward, done, truncated, info = env.step(action)
            episode_reward += reward
        print("EPISODE REWARD : ", episode_reward)
        episode_rewards.append(episode_reward)

    return episode_rewards

# Assuming 'model' and 'env' are properly defined and set up
mean_rewards = evaluate_model_2(model, env)
print("OKKKKKKKKKKKKKKKKKK")
# Plotting the results
import matplotlib.pyplot as plt
plt.plot(mean_rewards)
plt.xlabel('Episode')
plt.ylabel('Mean Reward')
plt.title('Evolution of Mean Reward')
plt.show()

Obs data: [ 0.10999452 -0.17288959  0.0670241  -0.10220852 -0.17995109  0.07120024]
(tensor([[-0.8153, -0.9058, -1.0265, -1.3047, -1.3843, -1.0039,  1.7462,  0.5357,
          1.5115, -0.0612, -0.6849, -0.1082, -0.5534, -0.0339, -1.1160, -1.3441,
         -0.5652,  0.9591, -0.6268,  0.9645,  0.3982,  1.8854, -1.1217,  0.4257,
         -1.8419,  0.2169,  0.1906, -1.1808,  0.6860, -1.0493,  0.4282,  1.0790,
         -0.5595, -1.1195, -1.4760,  1.4751,  0.8738, -0.6440,  1.0408]]), tensor([[-0.0643]]), tensor([-56.3544]))
JOINTS TO MOVE {'abs_z': -0.8153032, 'bust_y': -0.90579, 'bust_x': -1.0264974, 'r_shoulder_x': -1.3047128, 'r_shoulder_y': -1.38429, 'r_arm_z': -1.0038569, 'r_elbow_y': 1.7461916, 'l_shoulder_x': 0.5356631, 'l_shoulder_y': 1.5115496, 'l_arm_z': -0.061189976, 'l_elbow_y': -0.68485814, 'head_y': -0.108243704, 'head_z': -0.5533936}


C:\Users\org86\AppData\Local\Temp\ipykernel_26320\728585132.py:19: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_new.cpp:278.)
  obs_tensor = torch.tensor([obs_data], dtype=torch.float32)  # Add batch dimension


Obs data: [ 0.10545297 -0.17552817  0.0701278  -0.11319048 -0.18388875  0.07530705]
(tensor([[ 1.6258, -0.0618,  0.6941, -0.9339,  0.9724,  1.7549,  0.2277,  0.6569,
         -0.0482, -0.1940,  0.7184, -0.2599, -1.6179, -0.0775, -0.6937,  0.7441,
          0.2997,  1.4255,  1.1824, -0.2512,  0.9364,  0.3633, -0.9212,  0.8619,
          0.5994,  2.5237, -0.2683,  1.3739,  0.1704,  0.1409,  0.0351,  0.8559,
          0.4637, -0.2447, -0.3301, -0.4240, -1.3808,  0.1436, -1.2210]]), tensor([[-0.0643]]), tensor([-51.9467]))
JOINTS TO MOVE {'abs_z': 1.6258137, 'bust_y': -0.061755173, 'bust_x': 0.6940736, 'r_shoulder_x': -0.933931, 'r_shoulder_y': 0.9724219, 'r_arm_z': 1.7548583, 'r_elbow_y': 0.22773582, 'l_shoulder_x': 0.6569214, 'l_shoulder_y': -0.048179068, 'l_arm_z': -0.19399233, 'l_elbow_y': 0.7183671, 'head_y': -0.25985816, 'head_z': -1.617912}
Obs data: [ 0.11603431 -0.17327145  0.07092052 -0.0923724  -0.18023287  0.06689163]
(tensor([[-0.8254, -0.1621, -0.7200, -1.3898, -1.0821,  0.69

KeyboardInterrupt: 

In [ ]:
env.close()
from pypot import vrep
vrep.close_all_connections()